In [1]:
from basic_functions import data_load

# CONTINGENCY TABLE

In [2]:
#Load Datasets
X_train = data_load("data/X_train.csv")
y_train = data_load("data/y_train_clustered.csv")
X_test = data_load("data/X_test.csv")
y_test = data_load("data/y_test_clustered.csv")

Data loaded!
Data loaded!
Data loaded!
Data loaded!


In [4]:
#Delete household-size (not categorical)
X_train = X_train.copy()
X_test = X_test.copy()

In [6]:
#Pandas Crosstable (EMPLOYMENT STATUS)
import pandas as pd

table = pd.crosstab(X_train["federal_state"], y_train["income"], normalize="index")
table_rounded = (table*100).round(1)
print(table_rounded)

income         high  highest  lower middle  lowest  middle
federal_state                                             
0               9.2      3.6          47.2    14.3    25.8
1               4.4      1.1          42.4    37.8    14.3


# CHI²-TEST

In [8]:
from sklearn.feature_selection import chi2
chi2_scores, p_values = chi2(X_train, y_train)
chi2_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Chi2 Score": chi2_scores.round(2),
    "p-value": p_values.round(3)
}).sort_values(by="Chi2 Score", ascending=False)
print(chi2_df)

                      Feature  Chi2 Score  p-value
6                         job    37073.93    0.000
8                  livelihood     4473.57    0.000
10     highest_qualification      1288.00    0.000
1                      gender      569.85    0.000
9   educational_qualification      569.21    0.000
0               federal_state      526.89    0.000
7         employment_position      446.64    0.000
12     Household relationship      351.67    0.000
5           employment_sector      203.13    0.000
4           employment_status       62.76    0.000
2                 citizenship       60.55    0.000
11          primary_residence       56.97    0.000
3              marital_status        3.56    0.469


# CATEGORICAL NAIVE BAYESIAN

In [9]:
from sklearn.naive_bayes import CategoricalNB


clf = CategoricalNB()
clf.fit(X_train, y_train["income"])

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None
,min_categories,None


In [10]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = clf.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Genauigkeit (Accuracy): {accuracy:.2f}")

# Classification Report: Precision, Recall, F1-Score
print("Klassifikationsbericht:")
print(classification_report(y_test, y_pred))

Genauigkeit (Accuracy): 0.58
Klassifikationsbericht:
              precision    recall  f1-score   support

        high       0.32      0.41      0.36       350
     highest       0.38      0.41      0.40       139
lower middle       0.64      0.72      0.68      2072
      lowest       0.66      0.51      0.58       852
      middle       0.50      0.42      0.46      1034

    accuracy                           0.58      4447
   macro avg       0.50      0.49      0.49      4447
weighted avg       0.58      0.58      0.57      4447



In [18]:
import numpy as np

# Bayesian Prob: P(Federal_State (west, east) | income )

In [28]:
features = X_train.columns

probs = np.exp(clf.feature_log_prob_[0])

income_classes = clf.classes_
prob_df ={}

for i, label  in enumerate(income_classes):
    prob_df[label] = probs[i]

df = pd.DataFrame(prob_df)
print(df)


       high   highest  lower middle    lowest    middle
0  0.896149  0.928793      0.822758  0.611367  0.882666
1  0.103851  0.071207      0.177242  0.388633  0.117334


In [ ]:
west, east = X_train["federal_state"].value_counts(normalize=True)
lower_middle, middle, lowest, high, highest = (y_train["income"].value_counts(normalize=True))

In [45]:
p_lowestIncome_west = (df["lowest"][0] * lowest) / west
print(p_lowestIncome_west)

0.14248922061223365
